In [ ]:
"""
 * @file kitsune.ipynb
 * @author Leif Huender
 * @brief 
 * @version 0.1
 * @date 2024-01-07
 * 
 * @copyright Copyright (c) 2024 Leif Huender
 * Permission is hereby granted, free of charge, to any person obtaining a copy
 * of this software and associated documentation files (the "Software"), to deal
 * in the Software without restriction, including without limitation the rights
 * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the Software is
 * furnished to do so, subject to the following conditions:
 * 
 * The above copyright notice and this permission notice shall be included in
 * all copies or substantial portions of the Software.
 * 
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 * AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 * OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
 * THE SOFTWARE.
"""

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

print("All libraries successfully imported")

All libraries successfully imported


In [4]:
print(f'MPS is available: {torch.backends.mps.is_available()}')
print(f'CUDA is available: {torch.cuda.is_available()}')

MPS is available: True
CUDA is available: False


In [8]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))
    
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, stride):
        super(ResBlock, self).__init__()
        self.conv1 = ConvBlock(channels, channels, kernel_size, stride)
        self.conv2 = ConvBlock(channels, channels, kernel_size, stride)

    def forward(self,x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        out += residual
        return F.relu(out)
    
class ResTower(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, res_block_channels ,num_blocks):
        super(ResTower, self).__init__()
        self.initial_conv = ConvBlock(in_channels, out_channels, kernel_size, stride)
        self.res_blocks = nn.Sequential(*[ResBlock(res_block_channels, kernel_size, stride) for i in range(num_blocks)])
    def forward(self, x):
        x = self.initial_conv(x)
        x = self.res_blocks(x)
        return x

In [13]:
n_res_blocks = 39 # 19 or 39 to match alpha zero
model = ResTower(in_channels=256, out_channels=256, kernel_size=3, stride=3, res_block_channels=256, num_blocks=n_res_blocks)

In [14]:
total_params = sum(param.numel() for param in model.parameters())
print(f"Total number of parameters: {total_params}")


Total number of parameters: 46656768
